#### ver.2 
#### time_id kmeans
#### test_nn[target_name] = np.clip(test_nn[target_name], 0.00001, 0.5)

####  ver.1 Compare to PL=0.19945 (which was with train, test = read_train_test()  # missing (read_test=True, aug_vol=aug_vol) arguments
#### aug_vol = None (was 0.3 but with bug, missing arguments train, test = read_train_test() instead of (read_test=True, aug_vol=aug_vol) arguments

#### GroupKFold instead of KFold for LGBM  and KFold for NN
#### Stock to NoStock ratio = 0.60 : 
(0.3*test_predictions_nn  
'+ 0.7*(predictions_lgb*0.6+preds_lgb_nostock*0.4))
#### load_train_preprocessed = True 


  
  

  

### ver.5 use taus


### ver.3 No Taus for LGB, lgbs*0.6 + nn*0.4

### ver.2: np.clip('target'), np.inf to np.nan

### ver.1:
### blend: (lgb_w_Stock*0.7 + lgb_NoStock*03)*0.5 + nn*0.5


### =======================
### excluded stock_id from features in lgbm
### and set stock_embedding_size=2 (was 24) for NN

### - Joint traintest quantile transform, Normalize, lgb + NN



### Based on
### LB of lgbm+nn is 0.19758
### AlexiosLyon
https://www.kaggle.com/alexioslyon/lgbm-baseline

the stock embeddings part is a copy of 
### Douglas Marques https://www.kaggle.com/tatudoug/stock-embedding-ffnn-my-features,

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# from IPython.core.display import display, HTML
import time

import pandas as pd
import numpy as np # linear algebra
import numpy.matlib
import pandas as pd
import glob
import os
import gc

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import QuantileTransformer
# from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold, GroupKFold

import lightgbm as lgb

# import matplotlib.pyplot as plt
# import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.backend import sigmoid  # for Swish activation
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation

osj = os.path.join



platform = 'kaggle'  # 'local'  # 'kaggle'
experiment_name = 'blend_stock_NoStock lgb + NN'

debug = False  # True  # False
load_train_preprocessed = True  #  False  # False
path_train_preprocessed = '../input/optiver-train-preprocessed/train_processed.csv'

n_clusters_samples = 15  # 33
is_qtransform_separate = False
is_qtransform_joint = True
is_normalize_separate = False
is_normalize_joint = True
use_tau_lgb = True  #  False
is_train_lgb = True  # False  # True

aug_vol = None  # 0.30  # None  # 0.15   #  % increase volatility: None - if no augment 
aug_time_id_add = 1_000_000
target_name = 'target'

# n_folds for LGBM only (for nn it is 'nfolds')
n_folds = 5 if not debug else 2

scores_folds = {}

num_iter_lgb = 2000 if not debug else 50
n_epochs = 1200 if not debug else 4

# data directory
data_dir_dict = {
    'kaggle': '../input/optiver-realized-volatility-prediction/',
    'local': '../../data/raw/'
    }

out_dir_dict = {
    'kaggle': './',
    'local': '../../submissions/'}

data_dir = data_dir_dict[platform]
out_dir = out_dir_dict[platform]
del data_dir_dict, out_dir_dict

t0 = time.time()

def create_out_dir(cfg, prefix=''):
    datetime_str = time.strftime("%d_%m_time_%H_%M", time.localtime())
    out_dir = '../../submissions/{}_{}_m_{}_ep{}_bs{}_nf{}_t_{}'.format(
                prefix, experiment_name, datetime_str)   # bs, weight_decay, , folds_str,

    if cfg.debug:
        out_dir = osj(os.path.dirname(out_dir), 'debug_' + os.path.basename(out_dir))
    # models_outdir = osj(out_dir, 'models')
    os.makedirs(out_dir)
    # os.makedirs(models_outdir)
    return out_dir # , models_outdir

def normalize_joint(train: pd.DataFrame, test, feats: list, tfm=MinMaxScaler, feature_range=(-1,1)):
    if tfm == MinMaxScaler:
        scaler = tfm(feature_range=feature_range)
    else:
        scaler = tfm()
    len_train = train.shape[0]
    data = pd.concat([train[feats], test[feats]]).values
    data = scaler.fit_transform(data)
    train.loc[:, feats] = data[:len_train]
    test.loc[:, feats] = data[len_train:]
    return train, test

    
# Function to calculate first WAP
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# Function to calculate second WAP
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

def calc_wap3(df):
    wap = (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap4(df):
    wap = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

# Function to calculate the log of the return
# Remember that logb(x / y) = logb(x) - logb(y)
def log_return(series):
    return np.log(series).diff()

# Calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

# Function to read our base train and test set
def read_train_test(read_test=True, aug_vol=None):
    train = pd.read_csv(osj(data_dir, 'train.csv'))
    # Create a key to merge with book and trade data
    if aug_vol:
        train_aug = train.copy()
        train_aug.loc[:, 'target'] *= 1 + aug_vol
        train_aug.loc[:, 'time_id'] += aug_time_id_add
        train = pd.concat([train, train_aug]).reset_index(drop=True)
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)

    if read_test:
        test = pd.read_csv(osj(data_dir, 'test.csv'))
        test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    else:
        test = None
    print(f'Original train.csv has {train.shape[0]} rows')
    return train, test

def augment_book(df, aug_vol, book_or_trade='book'):
    df_aug = df.copy()
    if book_or_trade=='book':
        price_cols = ['bid_price1', 'ask_price1', 'bid_price2', 'ask_price2']
    else:
        price_cols = 'price'
    df_aug.loc[:, price_cols] += (df[price_cols] -1)*aug_vol
    df_aug.loc[:, 'time_id'] += aug_time_id_add
    df = pd.concat([df, df_aug]).reset_index(drop=True)
    return df
    
# Function to preprocess book data (for each stock id)
def book_preprocessor(file_path, is_train):
    df = pd.read_parquet(file_path)
            
    if debug and is_train:
        df = df.iloc[::16]  # sample(frac=0.1)
    
    # add augment rows
    if is_train and aug_vol:
        df = augment_book(df, aug_vol, book_or_trade='book')
    
    # Calculate Wap
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    df['wap3'] = calc_wap3(df)
    df['wap4'] = calc_wap4(df)
    # Calculate log returns
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    df['log_return3'] = df.groupby(['time_id'])['wap3'].apply(log_return)
    df['log_return4'] = df.groupby(['time_id'])['wap4'].apply(log_return)
    # Calculate wap balance
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    # Dict for aggregations
    create_feature_dict = {
        'wap1': [np.sum, np.std],
        'wap2': [np.sum, np.std],
        'wap3': [np.sum, np.std],
        'wap4': [np.sum, np.std],
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility],
        'log_return3': [realized_volatility],
        'log_return4': [realized_volatility],
        'wap_balance': [np.sum, np.max],
        'price_spread':[np.sum, np.max],
        'price_spread2':[np.sum, np.max],
        'bid_spread':[np.sum, np.max],
        'ask_spread':[np.sum, np.max],
        'total_volume':[np.sum, np.max],
        'volume_imbalance':[np.sum, np.max],
        "bid_ask_spread":[np.sum,  np.max],
    }
    create_feature_dict_time = {
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility],
        'log_return3': [realized_volatility],
        'log_return4': [realized_volatility],
    }
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)

    # Merge all
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id__100'], axis = 1, inplace = True)
    
    
    # Create row_id so we can merge
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature

# Function to preprocess trade data (for each stock id)
def trade_preprocessor(file_path, is_train):
    df = pd.read_parquet(file_path)
    if debug and is_train:
        df = df.iloc[::16]  # sample(frac=0.1)
    
    
    if is_train and aug_vol:
        df = augment_book(df, aug_vol, book_or_trade='trade')
    
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    df['amount']=df['price']*df['size']
    # Dict for aggregations
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, np.max, np.min],
        'order_count':[np.sum,np.max],
        'amount':[np.sum,np.max,np.min],
    }
    create_feature_dict_time = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.sum],
    }
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    

    # Get the stats for different windows
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)
    
    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff/price[1:])*100
        power = np.sum(val*vol[1:])
        return(power)
    
    lis = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]        
        tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)
        # new
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
        energy = np.mean(df_id['price'].values**2)
        iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
        
        # vol vars
        
        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values**2)
        iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)
        
        lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                   'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})
    
    df_lr = pd.DataFrame(lis)
        
   
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
    # Merge all
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id','time_id__100'], axis = 1, inplace = True)
    
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature

# Function to get group stats for the stock_id and time_id
def get_time_stock(df):
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_400', 'log_return2_realized_volatility_400', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_200', 'log_return2_realized_volatility_200', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_400', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_200']


    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the stock id
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df
    
# Funtion to make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, is_train = True):
    t0_prep = time.time()
    # Parrallel for loop
    def for_joblib(stock_id):
        # Train
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        # Preprocess book and trade data and merge them
        if len(glob.glob(osj(file_path_book, '*.parquet')))>0 and len(glob.glob(osj(file_path_trade, '*.parquet')))>0:
            df_tmp = pd.merge(book_preprocessor(file_path_book, is_train), trade_preprocessor(file_path_trade, is_train), on = 'row_id', how = 'left')
        else:
            df_tmp = None
                
        # Return the merge dataframe
        return df_tmp
    
    # Use parallel api to call paralle for loop
    if False:  # debug:
        df = []
        for stock_id in list_stock_ids:
            df0 = for_joblib(stock_id)
            df.append(df0)
    else: # not debug
        df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    
    df = [df_ for df_ in df if isinstance(df_, pd.DataFrame)]
    # Concatenate all the dataframes that return from Parallel
    df = pd.concat(df, ignore_index = True)
    print(f"reading def preprocessor time = {(time.time() - t0_prep)/60:.2f} min")
    return df

# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False



# Read train and test
train, test = read_train_test(read_test=True, aug_vol=aug_vol)

# Get unique stock ids 
train_stock_ids = train['stock_id'].unique()

In [ ]:
if load_train_preprocessed:
    train = pd.read_csv(path_train_preprocessed)
else:
    if debug:
        train_stock_ids = train_stock_ids[:4]
        train = train[train['stock_id'].isin(train_stock_ids)]
        test = test[test['stock_id'].isin(train_stock_ids)]

    # Preprocess them using Parallel and our single stock id functions
    train_ = preprocessor(train_stock_ids, is_train = True)
    train = train.merge(train_, on = ['row_id'], how = 'left')

#     # Get unique stock ids 
#     test_stock_ids = test['stock_id'].unique()
#     # Preprocess them using Parallel and our single stock id functions
#     test_ = preprocessor(test_stock_ids, is_train = False)
#     test = test.merge(test_, on = ['row_id'], how = 'left')

    # Get group stats of time_id and stock_id
    train = get_time_stock(train)
#     test = get_time_stock(test)

    # replace by order sum (tau)
    train['size_tau'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique'] )
#     test['size_tau'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique'] )
    #train['size_tau_450'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_450'] )
    #test['size_tau_450'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_450'] )
    train['size_tau_400'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_400'] )
#     test['size_tau_400'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_400'] )
    train['size_tau_300'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_300'] )
#     test['size_tau_300'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_300'] )
    #train['size_tau_150'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_150'] )
    #test['size_tau_150'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_150'] )
    train['size_tau_200'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_200'] )
#     test['size_tau_200'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_200'] )

    train['size_tau2'] = np.sqrt( 1/ train['trade_order_count_sum'] )
#     test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )
    #train['size_tau2_450'] = np.sqrt( 0.25/ train['trade_order_count_sum'] )
    #test['size_tau2_450'] = np.sqrt( 0.25/ test['trade_order_count_sum'] )
    train['size_tau2_400'] = np.sqrt( 0.33/ train['trade_order_count_sum'] )
#     test['size_tau2_400'] = np.sqrt( 0.33/ test['trade_order_count_sum'] )
    train['size_tau2_300'] = np.sqrt( 0.5/ train['trade_order_count_sum'] )
#     test['size_tau2_300'] = np.sqrt( 0.5/ test['trade_order_count_sum'] )
    #train['size_tau2_150'] = np.sqrt( 0.75/ train['trade_order_count_sum'] )
    #test['size_tau2_150'] = np.sqrt( 0.75/ test['trade_order_count_sum'] )
    train['size_tau2_200'] = np.sqrt( 0.66/ train['trade_order_count_sum'] )
#     test['size_tau2_200'] = np.sqrt( 0.66/ test['trade_order_count_sum'] )

    # delta tau
    train['size_tau2_d'] = train['size_tau2_400'] - train['size_tau2']
#     test['size_tau2_d'] = test['size_tau2_400'] - test['size_tau2']
    

# Get unique stock ids 
test_stock_ids = test['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')
test = get_time_stock(test)
test['size_tau'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique'] )
test['size_tau_400'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_400'] )
test['size_tau_300'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_300'] )
test['size_tau_200'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_200'] )
test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )
test['size_tau2_400'] = np.sqrt( 0.33/ test['trade_order_count_sum'] )
test['size_tau2_300'] = np.sqrt( 0.5/ test['trade_order_count_sum'] )
test['size_tau2_200'] = np.sqrt( 0.66/ test['trade_order_count_sum'] )
test['size_tau2_d'] = test['size_tau2_400'] - test['size_tau2']


train.replace([np.inf, -np.inf], np.nan, inplace=True)
test.replace([np.inf, -np.inf], np.nan,inplace=True)

train['target'] = np.clip(train['target'], 0.00001, 0.9)

print("train.shape, test.shape:", train.shape, test.shape)



In [ ]:
print("train.shape, test.shape:", train.shape, test.shape)

#### fillna before lgbm and normalize feats

In [ ]:
features_norm = [feat for feat in train.columns if feat not in ['stock_id','time_id',
                                                         'target','fold', 'row_id']]

feat_means = pd.concat([train[features_norm], test[features_norm]]).mean()

train[features_norm] = train[features_norm].fillna(feat_means)
test[features_norm] = test[features_norm].fillna(feat_means)

if is_normalize_joint:
    train, test = normalize_joint(train, test, features_norm, tfm=StandardScaler)


### time_id kmeans (add column with labels)

In [ ]:
%%time
kmeans_all = KMeans(n_clusters = n_clusters_samples, random_state = 24)


# train['kmeans_time_id'] = fit_predict(train[features_norm], sample_weight) = )
len_train = len(train)
traintest = pd.concat([train, test])
labels_ = kmeans_all.fit_predict(traintest[features_norm])  # , sample_weight) = )
train['kmeans_all_samples'] = labels_[:len_train]
test['kmeans_all_samples'] = labels_[len_train:]

del traintest, labels_, kmeans_all, features_norm; _ = gc.collect()

train['kmeans_all_samples'].value_counts()

In [ ]:
test['kmeans_all_samples'].head()

In [ ]:
colNames = [col for col in list(train.columns)
            if col not in {"stock_id", "time_id", "target", "row_id"}]

# making agg features

# train_p = pd.read_csv(osj(data_dir, 'train.csv'))
train_p, _ = read_train_test(read_test=False, aug_vol=aug_vol)
if debug:
    # train_stock_ids = train_stock_ids[:4]
    train_p = train_p[train_p['stock_id'].isin(train_stock_ids)]
    # test = test[test['stock_id'].isin(train_stock_ids)]

train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

# corrs of target values of 'stock_id's  (across the same 'time_id's):
corr = train_p.corr()

ids = corr.index  # stock_ids

n_clusters = 7 if not debug else 2
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(corr.values)
print("kmeans.labels_:", kmeans.labels_)

l = []
for n in range(n_clusters):
    l.append ( [ (x-1) for x in ( (ids+1)*(kmeans.labels_ == n)) if x > 0] )
    

mat = []
matTest = []

n = 0
for ind in l:
    print(f"kmean label indices: {ind}")
    newDf = train.loc[train['stock_id'].isin(ind) ]
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    mat.append ( newDf )
    
    newDf = test.loc[test['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append ( newDf )
    
    n+=1
    
mat1 = pd.concat(mat).reset_index()
mat1.drop(columns=['target'],inplace=True)

mat2 = pd.concat(matTest).reset_index()


mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])
mat1 = mat1.pivot(index='time_id', columns='stock_id')
mat1.columns = ["_".join(x) for x in mat1.columns.ravel()]
mat1.reset_index(inplace=True)

mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)

# todo: how he chose nnn (51) features out of 389 feats of mat1
nnn = ['time_id',
     'log_return1_realized_volatility_0c1',
     'log_return1_realized_volatility_1c1',     
     'log_return1_realized_volatility_3c1',
     'log_return1_realized_volatility_4c1',     
     'log_return1_realized_volatility_6c1',
     'total_volume_sum_0c1',
     'total_volume_sum_1c1', 
     'total_volume_sum_3c1',
     'total_volume_sum_4c1', 
     'total_volume_sum_6c1',
     'trade_size_sum_0c1',
     'trade_size_sum_1c1', 
     'trade_size_sum_3c1',
     'trade_size_sum_4c1', 
     'trade_size_sum_6c1',
     'trade_order_count_sum_0c1',
     'trade_order_count_sum_1c1',
     'trade_order_count_sum_3c1',
     'trade_order_count_sum_4c1',
     'trade_order_count_sum_6c1',      
     'price_spread_sum_0c1',
     'price_spread_sum_1c1',
     'price_spread_sum_3c1',
     'price_spread_sum_4c1',
     'price_spread_sum_6c1',   
     'bid_spread_sum_0c1',
     'bid_spread_sum_1c1',
     'bid_spread_sum_3c1',
     'bid_spread_sum_4c1',
     'bid_spread_sum_6c1',       
     'ask_spread_sum_0c1',
     'ask_spread_sum_1c1',
     'ask_spread_sum_3c1',
     'ask_spread_sum_4c1',
     'ask_spread_sum_6c1',   
     'volume_imbalance_sum_0c1',
     'volume_imbalance_sum_1c1',
     'volume_imbalance_sum_3c1',
     'volume_imbalance_sum_4c1',
     'volume_imbalance_sum_6c1',       
     'bid_ask_spread_sum_0c1',
     'bid_ask_spread_sum_1c1',
     'bid_ask_spread_sum_3c1',
     'bid_ask_spread_sum_4c1',
     'bid_ask_spread_sum_6c1',
     'size_tau2_0c1',
     'size_tau2_1c1',
     'size_tau2_3c1',
     'size_tau2_4c1',
     'size_tau2_6c1']
nnn = [col for col in nnn if col in mat1.columns]
train = pd.merge(train, mat1[nnn], how='left', on='time_id')
test = pd.merge(test, mat2[nnn], how='left', on='time_id')

del mat1,mat2
gc.collect()


# -----

seed0=2021
params0 = {
    'objective': 'rmse',
    'boosting_type': 'gbdt',
    'max_depth': -1,
    'max_bin':100,
    'min_data_in_leaf':500,
    'learning_rate': 0.05,
    'subsample': 0.72,
    'subsample_freq': 4,
    'feature_fraction': 0.5,
    'lambda_l1': 0.5,
    'lambda_l2': 1.0,
    'categorical_column':[0],
    'seed':seed0,
    'feature_fraction_seed': seed0,
    'bagging_seed': seed0,
    'drop_seed': seed0,
    'data_random_seed': seed0,
    'n_jobs':-1,
    'verbose': -1}
seed1=42
params1 = {
        'learning_rate': 0.1,        
        'lambda_l1': 2,
        'lambda_l2': 7,
        'num_leaves': 800,
        'min_sum_hessian_in_leaf': 20,
        'feature_fraction': 0.8,
        'feature_fraction_bynode': 0.8,
        'bagging_fraction': 0.9,
        'bagging_freq': 42,
        'min_data_in_leaf': 700,
        'max_depth': 4,
        'categorical_column':[0],
        'seed': seed1,
        'feature_fraction_seed': seed1,
        'bagging_seed': seed1,
        'drop_seed': seed1,
        'data_random_seed': seed1,
        'objective': 'rmse',
        'boosting': 'gbdt',
        'verbosity': -1,
        'n_jobs':-1,
    }
# Function to early stop with root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

def train_and_evaluate_lgb(train, test, params, drop_stock_id=True, seed=1):
    # Hyperparammeters (just basic)
#     idx_orig_train = train.index
    train = train.sample(frac=1.0, random_state = seed)
    params['seed'] = seed
    params['data_random_seed'] = seed
    features = [col for col in train.columns if col not in {"time_id", "target", "row_id"}]
    
    if drop_stock_id:
        features.remove('stock_id')
    if not use_tau_lgb:
        features = [col for col in features if '_tau' not in col]
    
    print(f"Num of features for LGB = {len(features)}")
        
    y = train['target']
    # Create out of folds array
    oof_predictions = np.zeros(train.shape[0])
    # Create test array to store predictions"
    test_predictions = np.zeros(test.shape[0])
    # Create a KFold object
#     kfold = KFold(n_splits = 5, random_state = 2021, shuffle = True)
    group_kfold = GroupKFold(n_splits = n_folds)
    # Iterate through each fold
    for fold, (trn_ind, val_ind) in enumerate(group_kfold.split(train['target'], groups=train['time_id'])):
        if aug_vol:
            val_nonaug_idx = train.iloc[val_ind][train['time_id']<aug_time_id_add].index
            x_train, x_val = train.iloc[trn_ind], train.iloc[val_ind].loc[val_nonaug_idx]
            y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind].loc[val_nonaug_idx]
        else:
            x_train, x_val = train.iloc[trn_ind], train.iloc[val_ind]
            y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        print(f'Training fold {fold + 1}, len(x_train)={len(x_train)}, len(x_val)={len(x_val)}')    
        # Root mean squared percentage error weights
        train_weights = 1 / np.square(y_train)
#         val_weights = 1 / np.square(y_val)
        train_dataset = lgb.Dataset(x_train[features], y_train, weight = train_weights)
        val_dataset = lgb.Dataset(x_val[features], y_val)  # , weight = val_weights)
        model = lgb.train(params = params,
                          num_boost_round=num_iter_lgb,
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          verbose_eval = 250,
                          early_stopping_rounds=300,
                          feval = feval_rmspe)
        # Add predictions to the out of folds array
        oof_predictions[val_ind] = model.predict(x_val[features])
        # Predict the test set        
        test_predictions += model.predict(test[features]) / n_folds
        
    lgb_rmspe_score = rmspe(y, oof_predictions)
    print(f'\nLGB OOF RMSPE score is {lgb_rmspe_score:.5f}\n')
    lgb.plot_importance(model,max_num_features=20)
    # Return test predictions
    return test_predictions, lgb_rmspe_score
# Traing and evaluate
if is_train_lgb:
    predictions_lgb, lgb_rmspe_score = train_and_evaluate_lgb(train, test, params0, drop_stock_id=False, seed=11)
    predictions_lgb2, lgb_rmspe_score2 = train_and_evaluate_lgb(train, test, params0, drop_stock_id=False, seed=22)
    predictions_lgb3, lgb_rmspe_score3 = train_and_evaluate_lgb(train, test, params0, drop_stock_id=False, seed=33)

    predictions_lgb = (predictions_lgb + predictions_lgb2 + predictions_lgb3) / 3
    del predictions_lgb2, predictions_lgb3
    
    preds_lgb_nostock, lgb_score_nostock = train_and_evaluate_lgb(train, test, params0, drop_stock_id=True, seed=44)
    preds_lgb_nostock2, lgb_score_nostock2 = train_and_evaluate_lgb(train, test, params0, drop_stock_id=True, seed=55)
    preds_lgb_nostock3, lgb_score_nostock3 = train_and_evaluate_lgb(train, test, params0, drop_stock_id=True, seed=66)

    preds_lgb_nostock = (preds_lgb_nostock + preds_lgb_nostock2 + preds_lgb_nostock3 ) / 3
    del preds_lgb_nostock2, preds_lgb_nostock3
    _ = gc.collect()
    print("lgb_rmspe_score, lgb_rmspe_score2, lgb_score_nostock, lgb_score_nostock2:\n", 
          lgb_rmspe_score, lgb_rmspe_score2, lgb_rmspe_score3, lgb_score_nostock, lgb_score_nostock2, lgb_score_nostock3)
#     test['target'] = predictions_lgb*0.7 + preds_nostock*0.3
    # test[['row_id', 'target']].to_csv(osj(out_dir, 'submission.csv'),index = False)

In [ ]:
print(train.shape, test.shape)
np.setxor1d(train.columns, test.columns)

In [ ]:
# --- FFNN ---

np.random.seed(42)
tf.random.set_seed(42)

def root_mean_squared_per_error(y_true, y_pred):
         return K.sqrt(K.mean(K.square( (y_true - y_pred)/ y_true )))
    
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=20, verbose=0,
    mode='min',restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=7, verbose=0,
    mode='min')

# kfold based on the knn++ algorithm

# out_train = pd.read_csv(osj(data_dir, 'train.csv'))
out_train, _ = read_train_test(read_test=False, aug_vol=aug_vol)

if debug:
    # train_stock_ids = train_stock_ids[:4]
    out_train = out_train[out_train['stock_id'].isin(train_stock_ids)]
    # test = test[test['stock_id'].isin(train_stock_ids)]

out_train = out_train.pivot(index='time_id', columns='stock_id', values='target')

#out_train[out_train.isna().any(axis=1)]
out_train = out_train.fillna(out_train.mean())

# code to add the just the read data after first execution

# data separation based on knn ++
nfolds = 5  # 5 # number of folds
index = []
totDist = []
values = []
# generates a matriz with the values of 
mat = out_train.values

scaler = MinMaxScaler(feature_range=(-1, 1))
mat = scaler.fit_transform(mat)

nind = int(mat.shape[0]/nfolds) # number of individuals

# adds index in the last column
mat = np.c_[mat,np.arange(mat.shape[0])]

# lineNumber = np.random.choice(np.array(mat.shape[0]), size=nfolds, replace=False)
lineNumber = np.random.choice(mat.shape[0], size=nfolds, replace=False)
lineNumber = np.sort(lineNumber)[::-1]

for n in range(nfolds):
    totDist.append(np.zeros(mat.shape[0]-nfolds))

# saves index
for n in range(nfolds):
    values.append([lineNumber[n]])    


s=[]
for n in range(nfolds):
    s.append(mat[lineNumber[n],:])
    
    mat = np.delete(mat, obj=lineNumber[n], axis=0)

for n in range(nind-1):    

    luck = np.random.uniform(0,1,nfolds)
    
    for cycle in range(nfolds):
         # saves the values of index           

        s[cycle] = np.matlib.repmat(s[cycle], mat.shape[0], 1)

        sumDist = np.sum( (mat[:,:-1] - s[cycle][:,:-1])**2 , axis=1)   
        totDist[cycle] += sumDist        
                
        # probabilities
        f = totDist[cycle]/np.sum(totDist[cycle]) # normalizing the totdist
        j = 0
        kn = 0
        for val in f:
            j += val        
            if (j > luck[cycle]): # the column was selected
                break
            kn +=1
        lineNumber[cycle] = kn
        
        # delete line of the value added    
        for n_iter in range(nfolds):
            
            totDist[n_iter] = np.delete(totDist[n_iter],obj=lineNumber[cycle], axis=0)
            j= 0
        
        s[cycle] = mat[lineNumber[cycle],:]
        values[cycle].append(int(mat[lineNumber[cycle],-1]))
        mat = np.delete(mat, obj=lineNumber[cycle], axis=0)


for n_mod in range(nfolds):
    values[n_mod] = out_train.index[values[n_mod]]

#colNames.remove('row_id')
train.replace([np.inf, -np.inf], np.nan, inplace=True)
test.replace([np.inf, -np.inf], np.nan,inplace=True)
qt_train = []
train_nn=train[colNames].copy()
test_nn=test[colNames].copy()

len_train = len(train_nn)
for col in colNames:
    qt = QuantileTransformer(random_state=21,n_quantiles=2000, output_distribution='normal')
    if is_qtransform_separate:
        train_nn[col] = qt.fit_transform(train_nn[[col]])
        test_nn[col] = qt.transform(test_nn[[col]])
    elif is_qtransform_joint:
        data = pd.concat([train_nn[[col]], test_nn[[col]]]).values
        data = qt.fit_transform(data)
        train_nn.loc[:,col] = data[:len_train]
        test_nn.loc[:, col] = data[len_train:]

train_nn[['stock_id','time_id','target']]=train[['stock_id','time_id','target']]
test_nn[['stock_id','time_id']]=test[['stock_id','time_id']]

# making agg features

# train_p = pd.read_csv(osj(data_dir, 'train.csv'))
train_p, _ = read_train_test(read_test=False, aug_vol=aug_vol)

if debug:
    # train_stock_ids = train_stock_ids[:4]
    train_p = train_p[train_p['stock_id'].isin(train_stock_ids)]
    # test = test[test['stock_id'].isin(train_stock_ids)]

train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()

ids = corr.index

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(corr.values)
print(kmeans.labels_)

l = []
for n in range(n_clusters):
    l.append ( [ (x-1) for x in ( (ids+1)*(kmeans.labels_ == n)) if x > 0] )
    

mat = []
matTest = []

n = 0
for ind in l:
    print(ind)
    newDf = train_nn.loc[train_nn['stock_id'].isin(ind) ]
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    mat.append ( newDf )
    
    newDf = test_nn.loc[test_nn['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append ( newDf )
    
    n+=1
    
mat1 = pd.concat(mat).reset_index()
mat1.drop(columns=['target'],inplace=True)

mat2 = pd.concat(matTest).reset_index()
mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])

nnn = ['time_id',
     'log_return1_realized_volatility_0c1',
     'log_return1_realized_volatility_1c1',     
     'log_return1_realized_volatility_3c1',
     'log_return1_realized_volatility_4c1',     
     'log_return1_realized_volatility_6c1',
     'total_volume_sum_0c1',
     'total_volume_sum_1c1', 
     'total_volume_sum_3c1',
     'total_volume_sum_4c1', 
     'total_volume_sum_6c1',
     'trade_size_sum_0c1',
     'trade_size_sum_1c1', 
     'trade_size_sum_3c1',
     'trade_size_sum_4c1', 
     'trade_size_sum_6c1',
     'trade_order_count_sum_0c1',
     'trade_order_count_sum_1c1',
     'trade_order_count_sum_3c1',
     'trade_order_count_sum_4c1',
     'trade_order_count_sum_6c1',      
     'price_spread_sum_0c1',
     'price_spread_sum_1c1',
     'price_spread_sum_3c1',
     'price_spread_sum_4c1',
     'price_spread_sum_6c1',   
     'bid_spread_sum_0c1',
     'bid_spread_sum_1c1',
     'bid_spread_sum_3c1',
     'bid_spread_sum_4c1',
     'bid_spread_sum_6c1',       
     'ask_spread_sum_0c1',
     'ask_spread_sum_1c1',
     'ask_spread_sum_3c1',
     'ask_spread_sum_4c1',
     'ask_spread_sum_6c1',   
     'volume_imbalance_sum_0c1',
     'volume_imbalance_sum_1c1',
     'volume_imbalance_sum_3c1',
     'volume_imbalance_sum_4c1',
     'volume_imbalance_sum_6c1',       
     'bid_ask_spread_sum_0c1',
     'bid_ask_spread_sum_1c1',
     'bid_ask_spread_sum_3c1',
     'bid_ask_spread_sum_4c1',
     'bid_ask_spread_sum_6c1',
     'size_tau2_0c1',
     'size_tau2_1c1',
     'size_tau2_3c1',
     'size_tau2_4c1',
     'size_tau2_6c1'] 

nnn = [col for col in nnn if col in mat1.columns]

mat1 = mat1.pivot(index='time_id', columns='stock_id')
mat1.columns = ["_".join(x) for x in mat1.columns.ravel()]
mat1.reset_index(inplace=True)

mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)

train_nn = pd.merge(train_nn,mat1[nnn],how='left',on='time_id')
test_nn = pd.merge(test_nn,mat2[nnn],how='left',on='time_id')
del mat1,mat2
if not debug: del train,test
gc.collect()

#https://bignerdranch.com/blog/implementing-swish-activation-function-in-keras/
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

get_custom_objects().update({'swish': Activation(swish)})

hidden_units = (128,64,32)
stock_embedding_size = 24

cat_data = train_nn['stock_id']

def base_model(num_input_dim=244):
    
    # Each instance will consist of two inputs: a single user id, and a single movie id
    stock_id_input = keras.Input(shape=(1,), name='stock_id')
    # num_input = keras.Input(shape=(244,), name='num_data')
    num_input = keras.Input(shape=(num_input_dim,), name='num_data')

    #embedding, flatenning and concatenating
    stock_embedded = keras.layers.Embedding(max(cat_data)+1, stock_embedding_size, 
                                           input_length=1, name='stock_embedding')(stock_id_input)
    stock_flattened = keras.layers.Flatten()(stock_embedded)
    out = keras.layers.Concatenate()([stock_flattened, num_input])
    
    # Add one or more hidden layers
    for n_hidden in hidden_units:

        out = keras.layers.Dense(n_hidden, activation='swish')(out)
        
    #out = keras.layers.Concatenate()([out, num_input])

    # A single output: our predicted rating
    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)
    
    model = keras.Model(
    inputs = [stock_id_input, num_input],
    outputs = out,
    )
    
    return model

# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [ ]:
target_name='target'
scores_folds = {}
model_name = 'NN'
pred_name = 'pred_{}'.format(model_name)

kf = model_selection.KFold(n_splits=nfolds, shuffle=True, random_state=2020)
scores_folds[model_name] = []
counter = 1

features_to_consider = list(train_nn)

features_to_consider.remove('time_id')
features_to_consider.remove('target')
try:
    features_to_consider.remove('pred_NN')
except:
    pass


train_nn[features_to_consider] = train_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())
test_nn[features_to_consider] = test_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())

train_nn[pred_name] = 0
test_nn[target_name] = 0
test_predictions_nn = np.zeros(test_nn.shape[0])

if is_normalize_joint:
    feats_x_stock_id = [feat for feat in features_to_consider if feat!='stock_id']
    train_nn, test_nn = normalize_joint(train_nn, test_nn, feats_x_stock_id)

for n_count in range(nfolds):
    
    indexes = np.arange(nfolds).astype(int)    
    indexes = np.delete(indexes,obj=n_count, axis=0) 
    
    indexes = np.r_[values[indexes[0]],values[indexes[1]],values[indexes[2]],values[indexes[3]]]
    
    X_train = train_nn.loc[train_nn.time_id.isin(indexes), features_to_consider]
    y_train = train_nn.loc[train_nn.time_id.isin(indexes), target_name]
    X_test = train_nn.loc[train_nn.time_id.isin(values[n_count])&(train_nn.time_id<aug_time_id_add), features_to_consider]
    y_test = train_nn.loc[train_nn.time_id.isin(values[n_count])&(train_nn.time_id<aug_time_id_add), target_name]
    print('Fold {}/{}, x_train.shape = {}, X_val.shape = {}'.format(counter, nfolds, X_train.shape, X_test.shape))

    #############################################################################################
    # NN
    #############################################################################################

    try:
        features_to_consider.remove('stock_id')
    except:
        pass

    num_data = X_train[features_to_consider]

    if not is_normalize_joint:
        scaler = MinMaxScaler(feature_range=(-1, 1))
        num_data = scaler.fit_transform(num_data.values)

    cat_data = X_train['stock_id']
    target = y_train

    num_data_test = X_test[features_to_consider]
    if not is_normalize_joint:
        num_data_test = scaler.transform(num_data_test.values)
    cat_data_test = X_test['stock_id']

    num_input_dim = num_data.shape[1]  #  + cat_data.shape[0]
    model = base_model(num_input_dim=num_input_dim)
    
    model.compile(
        keras.optimizers.Adam(learning_rate=0.006),
        loss=root_mean_squared_per_error
    )


    model.fit([cat_data, num_data], 
              target,               
              batch_size=2048,
              epochs=n_epochs,
              validation_data=([cat_data_test, num_data_test], y_test),
              callbacks=[es, plateau],
              validation_batch_size=len(y_test),
              shuffle=True,
             verbose = 2)

    preds = model.predict([cat_data_test, num_data_test]).reshape(1,-1)[0]
    score = round(rmspe(y_true = y_test, y_pred = preds),5)
    print('Fold {} {}: {}'.format(counter, model_name, score))
    scores_folds[model_name].append(score)

    if not is_normalize_joint:
        tt =scaler.transform(test_nn[features_to_consider].values)
    else:
        tt = test_nn[features_to_consider].values
    test_predictions_nn += model.predict([test_nn['stock_id'], tt]).reshape(1,-1)[0]/nfolds
       
    counter += 1
    features_to_consider.append('stock_id')


    
test_nn["row_id"] = test_nn["stock_id"].astype(str) + "-" + test_nn["time_id"].astype(str) 

if is_train_lgb:
#     test_nn[target_name] = (test_predictions_nn*0.7+preds_nn_nostock*0.3
    test_nn[target_name] = (0.3*test_predictions_nn
                            + 0.7*(predictions_lgb*0.6+preds_lgb_nostock*0.4))
else:
#     test_nn[target_name] = test_predictions_nn*0.7+preds_nn_nostock*0.3
    test_nn[target_name] = test_predictions_nn


score = round(rmspe(y_true = train_nn[target_name].values, y_pred = train_nn[pred_name].values), 5)
ave_score_folds = np.mean(scores_folds[model_name])

print('RMSPE {}: {} - Ave score: {} - Folds scores: {}'
            .format(model_name, score, ave_score_folds, scores_folds[model_name]))
if is_train_lgb:
    print(f'\nLGB OOF RMSPE score is {lgb_rmspe_score:.5f},\nLGB NoStock RMSPE score = {lgb_score_nostock:.5f}\n')

# display(test_nn[['row_id', target_name]].head(3))
test_nn[target_name] = np.clip(test_nn[target_name], 0.00001, 0.5)

test_nn[['row_id', target_name]].to_csv(osj(out_dir, 'submission.csv'), index = False)
#test[['row_id', target_name]].to_csv('submission.csv',index = False)
#kmeans N=5 [0.2101, 0.21399, 0.20923, 0.21398, 0.21175]

print(f"Total runtime = {(time.time()-t0)/60:.2f} min.")

In [ ]:
!ls

In [ ]:
preds.min(), preds.max(), preds.mean(), preds.shape

In [ ]:
[col for col in features_to_consider if 'kmeans' in col]